In [33]:
# #NOTE: this code is only used after completing 3_EntrainmentCalculation/Entrainment_Preprocessing.ipynb

In [34]:
####################################
#ENVIRONMENT SETUP

In [35]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [36]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [37]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, DataManager_Class

In [38]:
####################################
#LOADING CLASSES

In [49]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="LagrangianArrays", dataName="PROCESSED_Lagrangian_Binary_Array",
                                dtype='int16')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DC

In [40]:
####################################
#MAKING OUTPUT DIRECTORY

In [41]:
#OUTPUT DIRECTORIES
outputDirectory=os.path.join(mainDirectory,'Code','OUTPUT',
                             'Variable_Calculation','LagrangianArrays')
os.makedirs(outputDirectory, exist_ok=True)

#Data Output Directories
def MakeDataDirectories(outputDirectory,res,t_res,Nz_str):
    outputDataDirectory = os.path.join(outputDirectory,f"{res}_{t_res}_{Nz_str}nz",
                                       'PROCESSED_Lagrangian_Binary_Array')
    os.makedirs(outputDataDirectory, exist_ok=True)

    return outputDataDirectory

In [42]:
####################################
#FUNCTIONS

In [43]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str

In [57]:
def GetDataNC(ModelData,DataManager):
    filePath = DataManager.outputDataDirectory+f'/PROCESSED_Lagrangian_Binary_Array_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.h5'
    ds = xr.open_dataset(filePath, engine='h5netcdf')
    
    ds = ds.rename({'phony_dim_0': 'time', 'phony_dim_1': 'p'})

    print(f"Retrieved data from {filePath}")
    return ds

In [58]:
# def WriteTimesteps_NetCDF(dataNC, parcelNC, timeStrings, res,t_res,Np_str,Nz_str, outputDataDirectory,outputParcelDirectory):
        
#     for count,time in tqdm(enumerate(timeStrings), total=len(timeStrings), desc="Writing timesteps"):
#         dataT = dataNC.isel(time=count)
#         parcelT = parcelNC.isel(time=count)
    
#         #making filenames
#         outputDataFile = os.path.join(outputDataDirectory, f"cm1out_{res}_{t_res}_{Nz_str}nz_{time}.nc")
#         outputParcelFile = os.path.join(outputParcelDirectory, f"cm1out_pdata_{res}_{t_res}_{Np_str}np_{time}.nc")
        
#         dataT.to_netcdf(outputDataFile, engine="h5netcdf")
#         parcelT.to_netcdf(outputParcelFile, engine="h5netcdf")

# #EXAMPLE
# WriteTimesteps_NetCDF(dataNC, parcelNC, timeStrings, res, t_res, Np_str, Nz_str,outputDataDirectory,outputParcelDirectory)

In [59]:
def WriteTimesteps_H5(dataNC, timeStrings, res, t_res, Np_str, Nz_str,
                      outputDataDirectory):
    """
    Writes each timestep of dataNC and parcelNC to separate .h5 files,
    mirroring WriteTimestepsNetCDF but using h5py instead of xarray.to_netcdf.
    """
    from tqdm import tqdm

    for count, time in tqdm(enumerate(timeStrings), total=len(timeStrings), desc="Writing timesteps"):
        # Extract single timestep
        dataT = dataNC.isel(time=count)

        # Build file names (same as NetCDF version)
        outputDataFile = os.path.join(
            outputDataDirectory,
            f"PROCESSED_Lagrangian_Binary_Array_{res}_{t_res}_{Nz_str}nz_{time}.h5"
        )
        
        # --- Write data timestep ---
        with h5py.File(outputDataFile, "w") as f_data:
            for var_name, da in dataT.data_vars.items():
                f_data.create_dataset(
                    var_name,
                    data=da.values,
                    dtype="bool",
                    compression="gzip"
                )
                # optionally save metadata
                for attr, val in da.attrs.items():
                    f_data[var_name].attrs[attr] = val

        print(f"Saved time {time} to {outputDataFile}","\n")

In [60]:
####################################
#RUNNING

In [62]:
#getting data
dataNC = GetDataNC(ModelData,DataManager)

outputDataDirectory = MakeDataDirectories(outputDirectory,ModelData.res,ModelData.t_res,ModelData.Nz_str)

# running
WriteTimesteps_H5(dataNC, ModelData.timeStrings, ModelData.res, ModelData.t_res, ModelData.Np_str, ModelData.Nz_str,outputDataDirectory)

Retrieved data from /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_1e6.h5


Writing timesteps:   4%|▍         | 5/133 [00:00<00:03, 38.19it/s]

Saved time 0-00-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_0-00-00.h5 

Saved time 0-05-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_0-05-00.h5 

Saved time 0-10-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_0-10-00.h5 

Saved time 0-15-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_0

Writing timesteps:   7%|▋         | 9/133 [00:00<00:03, 38.32it/s]

Saved time 0-35-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_0-35-00.h5 

Saved time 0-40-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_0-40-00.h5 



...

Saved time 9-50-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_9-50-00.h5 

Saved time 9-55-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_9-55-00.h5 

Saved time 10-00-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-00-00.h5 

Saved time 10-05-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34n

Writing timesteps:  96%|█████████▌| 128/133 [00:05<00:00, 23.05it/s]

Saved time 10-20-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-20-00.h5 

Saved time 10-25-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-25-00.h5 

Saved time 10-30-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-30-00.h5 

Saved time 10-35-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min

Writing timesteps: 100%|██████████| 133/133 [00:05<00:00, 24.02it/s]

Saved time 10-45-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-45-00.h5 

Saved time 10-50-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-50-00.h5 

Saved time 10-55-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_10-55-00.h5 

Saved time 11-00-00 to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min